### Mission: To webscrap news articles from a variety of sources

- Use news API to identify interesting articles
- Build scrapping code to scrape the articles from the source provided by the news API
- Store the news articles in a permanent database
- Process the text of the articles in preparation for NLP
- Build interesting models around the text to understand sentiment

API source: https://newsapi.org/docs/endpoints/everything

In [140]:
import pandas as pd
from datetime import date
import requests
import time
now = time.time()

In [173]:
def save_news (search_terms,file_name, n_pagesize, start_page, end_pages, save_to_csv): 
    '''
    term_request = which is the key word for search.
    save_to_csv = True indicates csv will be saved
    '''
    
    # API requests
    for term in search_terms: 
        url = 'https://newsapi.org/v2/everything?'
        
        param = {
    #'country' : 'us',
    'q': term,  #search term 
    'apiKey' : 'e685d6e1420f4882b86d029ed3c1a11d',
    'pageSize': n_pagesize, #max page
    'language': 'en'}
        print (term)
        
        every_term = requests.get(url, params = param)
        #every_term.json()['articles']
        #every_term.json().keys()
        #print (every_term)

    articles = every_term.json()['articles'] 
    
    for page in range(start_page, end_pages): #go throught 10 times, and get more pages, 10 more pages
        param['page'] = page
        
        more_term = requests.get(url, params = param)
        more_term = more_term.json()['articles']
        
        #print(len(more_term))
        articles.extend(more_term)
    arts = pd.DataFrame(articles)
    
    # Drop null and duplicate 
    arts.dropna(inplace=True)
    arts.drop_duplicates(subset='content',inplace = True)
    
    # Create columns
    arts['source_id'] = arts['source'].map(lambda x: x['id'])
    arts['source_name'] = arts['source'].map(lambda x: x['name']) #break up the source, source id, and name colums seperate
    arts.drop (columns = ['source'], axis=1)
    arts['yes_disaster'] = 1

    # Save df to csv
    if save_to_csv == True: 
        arts.to_csv('../data/'+str(file_name)+'_'+str(now) +'.csv' ,index = False, sep = ",") #index = False for no extra columns
        print (f'{len(articles)} unique news haved been saved')

In [174]:
#call the function 
save_news (['disaster'], file_name ='b', n_pagesize=10, start_page=2, end_pages=3, save_to_csv=True)

disaster
20 unique news haved been saved


In [182]:
import glob
path = "../data/*.csv"
for fname in glob.glob(path):
    print (fname)

../data/b_1554939746.6693828.csv
../data/b_1554940839.5361001.csv


In [176]:
df= pd.read_csv('../data/b_1554939746.6693828.csv')
df = df.drop(['source','urlToImage','source_id'], axis=1)
df.head()

,author,content,description,publishedAt,title,url,source_name,yes_disaster
0,Nick Douglas,If I was setting up curriculum at a university...,“If I was setting up curriculum at a universit...,2019-02-21T18:00:00Z,How Tiny Mistakes Cause Huge Disasters,https://lifehacker.com/how-tiny-mistakes-cause...,Lifehacker.com,1
1,Megan Rose Dickey,The lack of diversity issue in Silicon Valley ...,The lack of diversity issue in Silicon Valley ...,2018-07-30T17:21:57Z,Venture capital’s diversity disaster,http://techcrunch.com/2018/07/30/venture-capit...,TechCrunch,1
2,Devin Coldewey,One would think that having one of the most hi...,One would think that having one of the most hi...,2019-03-08T20:32:24Z,MyEquifax.com is yet another security disaster,http://techcrunch.com/2019/03/08/myequifax-com...,TechCrunch,1
3,Matt Simon,California continues to take stock of its unpr...,"The rains are coming to California, and the fi...",2018-11-20T11:00:00Z,California Fire Survivors Brace for Debris-Fil...,https://www.wired.com/story/california-fire-su...,Wired,1
4,Nick Douglas,Does the Sydney Opera House really exist outsi...,Does the Sydney Opera House really exist outsi...,2018-05-14T13:00:00Z,"Tell Us Your Sydney, Australia Travel Tips",https://lifehacker.com/tell-us-your-sydney-aus...,Lifehacker.com,1
